# Part-05B: Streamlit Development with LangChain

## Introduction

This notebook focuses on developing a Streamlit application that leverages natural language processing (NLP) and machine learning models to provide interactive and dynamic visualizations and functionalities. By integrating LangChain and other NLP tools, we aim to create an engaging and informative user experience.

#### Objectives

1. **Prepare LLM-Related Elements**:
   - Load and preprocess the review and metadata for use within the Streamlit app.
   - Summarize reviews using pretrained models to provide concise insights.

2. **Implement ChatGPT Integration**:
   - Use LangChain’s `ChatOpenAI` to generate responses based on user queries.
   - Create custom prompts to perform specific tasks such as summarizing feedback or providing recommendations.

3. **Create a Vector Database**:
   - Split review texts into manageable chunks and embed them using OpenAI embeddings.
   - Store and retrieve document embeddings to perform similarity searches.

4. **Develop the Streamlit Application**:
   - Design and implement interactive components and visualizations to display review summaries, sentiment scores, and topic models.
   - Provide search and retrieval functionalities to answer user queries about the product.

By the end of this notebook, we will have a fully functional Streamlit application that allows users to interactively explore and analyze Amazon reviews, gaining valuable insights into customer preferences and sentiments.


### Saving Product Metadata

In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import pandas as pd

pd.set_option('display.max_columns',100)


##Load in the data
import json
with open("config/filepaths.json") as f:
    FPATHS = json.load(f)


import joblib
df = joblib.load(FPATHS['data']['processed-nlp']['processed-reviews-with-target_joblib'])
df

,asin,reviewerID,reviewText,summary,overall,year,title,brand,category,review-text-full,review-text-full_raw,tokens-dirty,tokens,lemmas,tokens-dirty-joined,tokens-joined,lemmas-joined,target-rating
0,B007JINB0W,A3Y51NV9HU5T2,"Great pasta taste and feel, but the spell in t...",Four Stars,4,2018,"Miracle Noodle Zero Carb, Gluten Free Shiratak...",Miracle Noodle,Grocery & Gourmet Food; Pasta & Noodles; Noodl...,"Four Stars: Great pasta taste and feel, but th...","Four Stars: Great pasta taste and feel, but th...","[four, stars, great, pasta, taste, and, feel, ...","[stars, great, pasta, taste, feel, spell, pack...","[star, great, pasta, taste, feel, spell, packa...",four stars great pasta taste and feel but the ...,stars great pasta taste feel spell packaged sk...,star great pasta taste feel spell package skrong,None
1,B007JINB0W,A3D7EFSRC6Y9MP,The texture just made it a little strange to e...,Okay but don't like texture,3,2014,"Miracle Noodle Zero Carb, Gluten Free Shiratak...",Miracle Noodle,Grocery & Gourmet Food; Pasta & Noodles; Noodl...,Okay but don't like texture: The texture just ...,Okay but don't like texture: The texture just ...,"[okay, but, do, n't, like, texture, the, textu...","[okay, like, texture, texture, little, strange...","[okay, like, texture, texture, little, strange...",okay but do n't like texture the texture just ...,okay like texture texture little strange eat f...,okay like texture texture little strange eat f...,None
2,B007JINB0W,A4AM5KBP3I2R,The herb flavor makes the odd texture of shira...,Go for the green noodles,5,2018,"Miracle Noodle Zero Carb, Gluten Free Shiratak...",Miracle Noodle,Grocery & Gourmet Food; Pasta & Noodles; Noodl...,Go for the green noodles: The herb flavor make...,Go for the green noodles: The herb flavor make...,"[go, for, the, green, noodles, the, herb, flav...","[green, noodles, herb, flavor, makes, odd, tex...","[green, noodle, herb, flavor, make, odd, textu...",go for the green noodles the herb flavor makes...,green noodles herb flavor makes odd texture sh...,green noodle herb flavor make odd texture shir...,High
3,B007JINB0W,A3GHK4IL78DB7Y,I didn't have a problem at all with a half fil...,Its an awesome substitute.,5,2018,"Miracle Noodle Zero Carb, Gluten Free Shiratak...",Miracle Noodle,Grocery & Gourmet Food; Pasta & Noodles; Noodl...,Its an awesome substitute.: I didn't have a pr...,Its an awesome substitute.: I didn't have a pr...,"[its, an, awesome, substitute, i, did, n't, ha...","[awesome, substitute, problem, half, filled, b...","[awesome, substitute, problem, half, fill, bag...",its an awesome substitute i did n't have a pro...,awesome substitute problem half filled bag use...,awesome substitute problem half fill bag user ...,High
4,B007JINB0W,AH3B94LQOPPY6,They taste like whatever you cook them with.,Five Stars,5,2016,"Miracle Noodle Zero Carb, Gluten Free Shiratak...",Miracle Noodle,Grocery & Gourmet Food; Pasta & Noodles; Noodl...,Five Stars: They taste like whatever you cook ...,Five Stars: They taste like whatever you cook ...,"[five, stars, they, taste, like, whatever, you...","[stars, taste, like, cook]","[star, taste, like, cook]",five stars they taste like whatever you cook t...,stars taste like cook,star taste like cook,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4358,B007JINB0W,A73IG1ED6S0JR,Product arrived with two of the bags punctured...,would not recomend,1,2016,"Miracle Noodle Zero Carb, Gluten Free Shiratak...",Miracle Noodle,Grocery & Gourmet Food; Pasta & Noodles; Noodl...,would not recomend: Product arrived with two o...,would not recomend: Product arrived with two o...,"[would, not, recomend, product, arrived, with,...","[recomend, product, arrived, bags, punctured, ...","[recomend, product, arrive, bag, puncture, sme...",would not recomend product arrived with two of...,recomend product arrived bags punctured smells...,recomend product arrive bag puncture smell bad,Low
4359,B007JINB0W,A1XZ2H0MYG54M0,Ok.,Fiv

In [3]:
meta_df = pd.read_csv(FPATHS['data']['subset']['metadata_csv'])
meta_df.head()

,asin,category,description,title,brand,feature,rank,main_cat,price,imageURL,imageURLHighRes,details,Category_Beverages,"Category_Bottled Beverages, Water & Drink Mixes",Category_Candy & Chocolate,"Category_Canned, Jarred & Packaged Foods",Category_Coffee,"Category_Coffee, Tea & Cocoa",Category_Cooking & Baking,Category_Grocery & Gourmet Food,"Category_Herbs, Spices & Seasonings","Category_Sauces, Gravies & Marinades",Category_Snack Foods,Category_Tea,category_list
0,B00BUKL666,Grocery & Gourmet Food; Snack Foods; Bars; Nut...,'These bars are where our journey started and ...,"KIND Bars, Dark Chocolate Nuts &amp; Sea Salt,...",KIND,NaN,18 in Grocery & Gourmet Food (,Grocery,$13.67,'https://images-na.ssl-images-amazon.com/image...,'https://images-na.ssl-images-amazon.com/image...,{'\\n Product Dimensions: \\n ': '6.8 x ...,0,0,0,0,0,0,0,1,0,0,1,0,"['Grocery & Gourmet Food', 'Snack Foods', 'Bar..."
1,B008QMX2SG,Grocery & Gourmet Food; Snack Foods; Bars; Nut...,'These bars are where our journey started and ...,"Kind Bars, Madagascar Vanilla Almond, Gluten F...",KIND,NaN,"2,949 in Grocery & Gourmet Food (",Grocery,$14.79,'https://images-na.ssl-images-amazon.com/image...,'https://images-na.ssl-images-amazon.com/image...,{'\\n Product Dimensions: \\n ': '2 x 4 ...,0,0,0,0,0,0,0,1,0,0,1,0,"['Grocery & Gourmet Food', 'Snack Foods', 'Bar..."
2,B00D3M2QP4,Grocery & Gourmet Food; Breakfast Foods; Break...,'These bars are where our journey started and ...,"KIND Bars, Dark Chocolate Chili Almond, Gluten...",KIND,NaN,"4,575 in Grocery & Gourmet Food (",Grocery,$15.53,'https://images-na.ssl-images-amazon.com/image...,'https://images-na.ssl-images-amazon.com/image...,{'\\n Product Dimensions: \\n ': '7 x 6 ...,0,0,0,0,0,0,0,1,0,0,0,0,"['Grocery & Gourmet Food', 'Breakfast Foods', ..."
3,B00542YXFW,"Grocery & Gourmet Food; Beverages; Coffee, Tea...","'Mild, but round and full licorice flavor and ...","Davidson's Tea Bulk, Anise Seed, 16-Ounce Bag",Davidson's Tea,NaN,"100,853 in Grocery & Gourmet Food (",Grocery,$15.00,'https://images-na.ssl-images-amazon.com/image...,'https://images-na.ssl-images-amazon.com/image...,"{'Shipping Weight:': '1.1 pounds (', 'Domestic...",1,0,0,0,0,1,0,1,0,0,0,1,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."
4,B000F4DKAI,"Grocery & Gourmet Food; Beverages; Coffee, Tea...",'First started as much needed refreshment betw...,Twinings of London English Afternoon Black Tea...,Twinings,NaN,"19,796 in Grocery & Gourmet Food (",Grocery,$23.70,'https://images-na.ssl-images-amazon.com/image...,'https://images-na.ssl-images-amazon.com/image...,"{'Shipping Weight:': '1.3 pounds (', 'Domestic...",1,0,0,0,0,1,0,1,0,0,0,1,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."


In [4]:
# Filter for only products in the reivews
product_metadata = meta_df[meta_df['asin'].isin(df['asin'].unique())]
# product_metadata = product_metadata.rename({'category_list':'categories'},axis=1)

product_metadata = product_metadata.reset_index(drop=True)
product_metadata

,asin,category,description,title,brand,feature,rank,main_cat,price,imageURL,imageURLHighRes,details,Category_Beverages,"Category_Bottled Beverages, Water & Drink Mixes",Category_Candy & Chocolate,"Category_Canned, Jarred & Packaged Foods",Category_Coffee,"Category_Coffee, Tea & Cocoa",Category_Cooking & Baking,Category_Grocery & Gourmet Food,"Category_Herbs, Spices & Seasonings","Category_Sauces, Gravies & Marinades",Category_Snack Foods,Category_Tea,category_list
0,B007JINB0W,Grocery & Gourmet Food; Pasta & Noodles; Noodl...,'Nutrition Facts Serving Size: 3 oz Servings P...,"Miracle Noodle Zero Carb, Gluten Free Shiratak...",Miracle Noodle,"'<span class=""a-size-base a-color-secondary"">\...","119,683 in Grocery & Gourmet Food (",Grocery,$59.76,'https://images-na.ssl-images-amazon.com/image...,'https://images-na.ssl-images-amazon.com/image...,"{'\\n Item Weight: \\n ': '3.07 pounds',...",0,0,0,0,0,0,0,1,0,0,0,0,"['Grocery & Gourmet Food', 'Pasta & Noodles', ..."


In [5]:
# fpath_df = FPATHS['data']['processed-nlp']['processed-reviews-with-target_json']
# df = pd.read_json(fpath_df)
# df.head()

### Convert df into df_llm and replace df

In [6]:
df['target-rating'].value_counts(dropna=False)

High    1868
Low     1437
None    1058
Name: target-rating, dtype: int64

In [7]:
# df_llm = df.dropna(subset=['target-rating'])
llm_cols = ['reviewerID','review-text-full','overall','target-rating']
df_llm = df.loc[:,#df['target-rating'].notna(),
                llm_cols]
df_llm = df_llm.rename({'overall':'stars','review-text-full':'review',
                       'target-rating':'group'},axis=1)
df_llm

,reviewerID,review,stars,group
0,A3Y51NV9HU5T2,"Four Stars: Great pasta taste and feel, but th...",4,None
1,A3D7EFSRC6Y9MP,Okay but don't like texture: The texture just ...,3,None
2,A4AM5KBP3I2R,Go for the green noodles: The herb flavor make...,5,High
3,A3GHK4IL78DB7Y,Its an awesome substitute.: I didn't have a pr...,5,High
4,AH3B94LQOPPY6,Five Stars: They taste like whatever you cook ...,5,High
...,...,...,...,...
4358,A73IG1ED6S0JR,would not recomend: Product arrived with two o...,1,Low
4359,A1XZ2H0MYG54M0,Five Stars: Ok.,5,High
4360,A3I2YF0MXB7P0B,"Not awful, but now I know why these were on sa...",2,Low
4361,A2UELLFLITPMT1,Don't even try it.: Truly horrific. Like eatin...,1,Low


In [8]:
dup_subset = ['review','stars']
df_llm.duplicated(subset=dup_subset).sum()

42

In [9]:
# 
fpath_llm = FPATHS['data']['app']['reviews-with-target-for-llm_csv']
df_llm = df_llm.drop_duplicates(subset=dup_subset)
df_llm.to_csv(fpath_llm, index=False)

In [10]:
# df_llm.duplicated(subset=['review','stars']).sum()

In [11]:
import pandas as pd
df = pd.read_csv(fpath_llm)
df['stars'].value_counts()

5    1838
1    1061
4     605
3     451
2     366
Name: stars, dtype: int64

In [12]:
df.duplicated(subset=dup_subset).sum()

0

In [13]:
import streamlit as st
import streamlit.components.v1 as components
import pandas as pd
import numpy as np
import os
import joblib
import tensorflow as tf
from PIL import Image

%load_ext autoreload
%autoreload 2
    
import custom_functions as fn

# # Get Fpaths
# @st.cache_data
# def get_app_fpaths(fpath='config/filepaths.json'):
# 	import json
# 	with open(fpath ) as f:
# 		return json.load(f)



##Load in the data
import json
with open("config/filepaths.json") as f:
    FPATHS = json.load(f)
    
# st.header("Exploratory Data Analysis of Amazon Reviews ")

# if st.checkbox('[Dev] Show FPATHS?',value=False):
#     FPATHS
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os,json

# with open("/Users/codingdojo/.secret/open-ai.json") as f:
    # creds = json.load(f)

# os.environ['OPENAI_API_KEY'] = creds['api-key']

In [15]:
# @st.cache_data    
def load_df(fpath):
    if fpath.endswith(".joblib"):
        import joblib
        return joblib.load(fpath)
    elif fpath.endswith('.csv'):
        import pandas as pd
    return pd.read_csv(fpath)

# @st.cache_data
def load_metadata(fpath):
    import pandas as pd
    return pd.read_json(fpath)

# df = load_df(FPATHS['data']['processed-nlp']['processed-reviews-with-target_joblib'])
df = load_df(fpath = FPATHS['data']['app']['reviews-with-target-for-llm_csv'])

meta_df = load_metadata(FPATHS['data']['app']['product-metadata_json'])
product= meta_df.iloc[0]
product

Title            Miracle Noodle Zero Carb\n Gluten Free Shirata...
Description      Nutrition Facts Serving Size: 3 oz Servings Pe...
Brand                                               Miracle Noodle
Price                                                       $59.76
Rank                             119,683 in Grocery & Gourmet Food
Categories       [Grocery & Gourmet Food, Pasta & Noodles, Nood...
Product Image     images/selected-products/miracle-noodle-2024.jpg
Image Files      [images/selected-products/51RZohgUHBL.jpg, ima...
Title (Raw)      Miracle Noodle Zero Carb, Gluten Free Shiratak...
Name: B007JINB0W, dtype: object

In [16]:

# product_json  ={'Title':product.loc['Title (Raw)'],
#                'Brand':product.loc['Brand'],
#                "Price":product.loc['Price'],
#                "Categories": "; ".join(product.loc['Categories']),
#                 'ProductID':product.name
#                }
# product_json

In [17]:
print(product['Description'])

Nutrition Facts Serving Size: 3 oz Servings Per Container: 2.3 Amount Per Serving Calories 0 Calories from Fat 0 Total Fat 0g 0% Protein 0g 0% Protein <1g 0% Sugar 0g 0% Carbohydrate <1g - only fiber Zero Net Carbs, Zero Calories, Zero Glycemic Index Ingredients: Water, glucomannan (soluble fiber), calcium additive. No soy, gluten, or wheat! Kosher', 'Manufactured in a facility that also processes soy products. . This product is labelled to United States standards and may differ from similar products sold elsewhere in its ingredients, labeling and allergen warnings', 'No soy, gluten or wheat', 'Water, glucomannan (fiber), Spinach Powder, calcium additive.', 'Cooking instructions: drain the water out of the package and rinse the noodles in cool water for 10-15 seconds. Blanch in boiling water for about a minute. Place on a paper towel to dry and then add to your favorite dish.<br>Refrigerate after opening. Do not freeze.', 'Statements regarding dietary supplements have not been evaluate

In [18]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.1)
response = chat.invoke(f"Convert this raw text into a formatted nutrition table:\n\n {product['Description']}")
response

AIMessage(content='| Nutrition Facts              |              |\n|------------------------------|--------------|\n| Serving Size                 | 3 oz         |\n| Servings Per Container        | 2.3          |\n|------------------------------|--------------|\n| Amount Per Serving           |              |\n| Calories                     | 0            |\n| Calories from Fat            | 0            |\n| Total Fat                    | 0g           |\n| Protein                      | 0g           |\n| Protein                      | <1g          |\n| Sugar                        | 0g           |\n| Carbohydrate                 | <1g          |\n| Fiber                        | Zero         |\n| Net Carbs                    | Zero         |\n| Calories                     | Zero         |\n| Glycemic Index               | Zero         |\n|------------------------------|--------------|\n| Ingredients                  |              |\n| Water                        |              |\n

In [19]:
print(response.content)

| Nutrition Facts              |              |
|------------------------------|--------------|
| Serving Size                 | 3 oz         |
| Servings Per Container        | 2.3          |
|------------------------------|--------------|
| Amount Per Serving           |              |
| Calories                     | 0            |
| Calories from Fat            | 0            |
| Total Fat                    | 0g           |
| Protein                      | 0g           |
| Protein                      | <1g          |
| Sugar                        | 0g           |
| Carbohydrate                 | <1g          |
| Fiber                        | Zero         |
| Net Carbs                    | Zero         |
| Calories                     | Zero         |
| Glycemic Index               | Zero         |
|------------------------------|--------------|
| Ingredients                  |              |
| Water                        |              |
| Glucomannan (soluble fiber)  |       

### Save Product Info 

In [20]:
# Create product info 
response = chat.invoke(f"Convert this raw text into a formatted nutrition table:\n\n {product['Description']}")

product_json  ={'Title':product.loc['Title (Raw)'],
               'Brand':product.loc['Brand'],
               "Price":product.loc['Price'],
               "Categories": "; ".join(product.loc['Categories']),
                'ProductID':product.name,
                'Description':response.content
               }
product_json

{'Title': 'Miracle Noodle Zero Carb, Gluten Free Shirataki Pasta, Spinach Angel Hair, 7-Ounce (Pack of 24)',
 'Brand': 'Miracle Noodle',
 'Price': '$59.76',
 'Categories': 'Grocery & Gourmet Food; Pasta & Noodles; Noodles; Shirataki',
 'ProductID': 'B007JINB0W',
 'Description': '| Nutrition Facts            | Amount Per Serving |\n|---------------------------|---------------------|\n| Serving Size: 3 oz        |                     |\n| Servings Per Container: 2.3|                     |\n| Calories                  | 0                   |\n| Calories from Fat         | 0                   |\n| Total Fat                 | 0g 0%              |\n| Protein                   | 0g 0%              |\n| Protein                   | <1g 0%             |\n| Sugar                     | 0g 0%              |\n| Carbohydrate              | <1g - only fiber    |\n|                           | Zero Net Carbs, Zero Calories, Zero Glycemic Index |\n| Ingredients               | Water, glucomannan (solubl

- Changing product_json to string.



In [21]:
product_string = "Product Info:\n"
for k,v in product_json.items():
    product_string+=f"\n{k} = {v}\n"
print(product_string)

Product Info:

Title = Miracle Noodle Zero Carb, Gluten Free Shirataki Pasta, Spinach Angel Hair, 7-Ounce (Pack of 24)

Brand = Miracle Noodle

Price = $59.76

Categories = Grocery & Gourmet Food; Pasta & Noodles; Noodles; Shirataki

ProductID = B007JINB0W

Description = | Nutrition Facts            | Amount Per Serving |
|---------------------------|---------------------|
| Serving Size: 3 oz        |                     |
| Servings Per Container: 2.3|                     |
| Calories                  | 0                   |
| Calories from Fat         | 0                   |
| Total Fat                 | 0g 0%              |
| Protein                   | 0g 0%              |
| Protein                   | <1g 0%             |
| Sugar                     | 0g 0%              |
| Carbohydrate              | <1g - only fiber    |
|                           | Zero Net Carbs, Zero Calories, Zero Glycemic Index |
| Ingredients               | Water, glucomannan (soluble fiber), calcium ad

In [22]:
import json
with open(FPATHS['data']['app']['product-metadata-llm_json'],'w') as f:
    json.dump(product_json, f)

In [23]:
with open(FPATHS['data']['app']['product-metadata-llm_json'],'r') as f:
    loaded_prod_json = json.load(f)
loaded_prod_json

{'Title': 'Miracle Noodle Zero Carb, Gluten Free Shirataki Pasta, Spinach Angel Hair, 7-Ounce (Pack of 24)',
 'Brand': 'Miracle Noodle',
 'Price': '$59.76',
 'Categories': 'Grocery & Gourmet Food; Pasta & Noodles; Noodles; Shirataki',
 'ProductID': 'B007JINB0W',
 'Description': '| Nutrition Facts            | Amount Per Serving |\n|---------------------------|---------------------|\n| Serving Size: 3 oz        |                     |\n| Servings Per Container: 2.3|                     |\n| Calories                  | 0                   |\n| Calories from Fat         | 0                   |\n| Total Fat                 | 0g 0%              |\n| Protein                   | 0g 0%              |\n| Protein                   | <1g 0%             |\n| Sugar                     | 0g 0%              |\n| Carbohydrate              | <1g - only fiber    |\n|                           | Zero Net Carbs, Zero Calories, Zero Glycemic Index |\n| Ingredients               | Water, glucomannan (solubl

In [24]:
# df = df.dropna(subset='target-rating')
df.head(1)

,reviewerID,review,stars,group
0,A3Y51NV9HU5T2,"Four Stars: Great pasta taste and feel, but th...",4,NaN


In [25]:
# display(meta_df)
# df.head()

In [26]:
import plotly.express as px
import plotly.io as pio
pio.templates.default=None

## Summarizing Reviews Using Pretrained BART from HuggingFace

- The summaries will be displayed for the user but also used as context for Chat-GPT Recommendations.

In [27]:
RUN_SUMMARIZATION_CODE = True

In [28]:
# df = df.drop_duplicates(subset=)
df.duplicated(subset=dup_subset).sum()

0

In [29]:
%%time
# Load model directly
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

CPU times: user 714 ms, sys: 218 ms, total: 932 ms
Wall time: 1.24 s


In [30]:
model_name = "kabita-choudhary/finetuned-bart-for-conversation-summary"
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download $model_name

/Users/codingdojo/.cache/huggingface/hub/models--kabita-choudhary--finetuned-bart-for-conversation-summary/snapshots/702abeb4d99b5255c099344410cc729892433490


In [ ]:
%%time

# if RUN_SUMMARIZATION_CODE:
# model_name = "kabita-choudhary/finetuned-bart-for-conversation-summary"
model_fpath ="/Users/codingdojo/.cache/huggingface/hub/models--kabita-choudhary--finetuned-bart-for-conversation-summary/snapshots/702abeb4d99b5255c099344410cc729892433490/"
tokenizer_a = AutoTokenizer.from_pretrained(model_fpath)
model_a = AutoModelForSeq2SeqLM.from_pretrained(model_fpath)
print("Model downloaded...")

In [ ]:
if RUN_SUMMARIZATION_CODE:

        # Getting group texts
    grp_idx_dict = df.groupby('group').groups
    # high_promt_prefix = "Summarize what customers likeed about this product:"
    
    high_data  = "\n".join(df.loc[grp_idx_dict['High'], 'review'])
    
    # low_promt_prefix = "Summarize what customers did not like about this product:"
    low_data = "\n".join(df.loc[grp_idx_dict['Low'], 'review'])



In [ ]:
%%time
if RUN_SUMMARIZATION_CODE:

    # Tokenizing for PyTorch
    tokenizer_params = dict(truncation=True, return_tensors='pt', max_length=1024,
                          padding='max_length')
    low_tokens_a = tokenizer_a(low_data,**tokenizer_params)
    
    high_tokens_a = tokenizer_a(high_data, **tokenizer_params)

In [ ]:
%%time
if RUN_SUMMARIZATION_CODE:
    
    shared_params = dict( num_beams=6,
        max_length=300,
        min_length=125,
        length_penalty=2.0,
        early_stopping=True,
                         no_repeat_ngram_size=3,
        # temperature=0.1, do_sample=True,
                        )
    low_summary_ids_a = model_a.generate(
        low_tokens_a["input_ids"], **shared_params)
    high_summary_ids_a = model_a.generate(
        high_tokens_a["input_ids"], **shared_params)

In [ ]:
%%time
if RUN_SUMMARIZATION_CODE:

    summary_low_a = tokenizer_a.decode(low_summary_ids_a[0], skip_special_tokens=True)
    print(summary_low_a)


In [ ]:
if RUN_SUMMARIZATION_CODE:

    summary_high_a = tokenizer_a.decode(high_summary_ids_a[0], skip_special_tokens=True)
    print(summary_high_a)


In [ ]:
%%time
if RUN_SUMMARIZATION_CODE:
    # model_results = {'model-info':{'model-name':model_name,
    #                                  'model-params':shared_params,
    #                                'tokenizer-params':tokenizer_params},
                     
    #                   'summary-high':summary_high_a,
    #                   'summary-low':summary_low_a}
    # print(model_results)
    model_results = {'model-info':{'model-name':model_name,
                                     'model-params':shared_params,
                                   'tokenizer-params':tokenizer_params},
                     'summaries':{'high':summary_high_a,
                                 'low', summary_low_a}
                    }
                     
                      # 'summary-high':summary_high_a,
                      # 'summary-low':summary_low_a}
    print(model_results)

In [ ]:
model_results

In [ ]:
if RUN_SUMMARIZATION_CODE:
    
    import json
    fname_summaries = FPATHS['results']['review-summary-01_json']
    print(fname_summaries)

In [ ]:
if RUN_SUMMARIZATION_CODE:
    
    # answer  = input("Save these results?")
    # if answer.lower().startswith('y'):
    with open(fname_summaries,'w') as f:
        json.dump(model_results, f )

In [ ]:
if RUN_SUMMARIZATION_CODE:
    
    # Getting entire text
    combined_data = '\n\n'.join(df['review-text-full'])

    
    combined_tokens = tokenizer_a(combined_data, truncation=True, return_tensors='pt', #max_length=2056,
                          padding='max_length')

In [ ]:
%%time
if RUN_SUMMARIZATION_CODE:
    
    shared_params = dict( num_beams=6,
        max_length=1000,
        min_length=300,
        length_penalty=2.0,
                         no_repeat_ngram_size=3,
    
        # early_stopping=True,
        # temperature=0.1, do_sample=True,
                        )
    combined_summary_ids = model_a.generate( combined_tokens["input_ids"], **shared_params)

In [ ]:
if RUN_SUMMARIZATION_CODE:
    
    summary_combined = tokenizer_a.decode(combined_summary_ids[0], skip_special_tokens=True)
    print(summary_combined)


In [ ]:
if RUN_SUMMARIZATION_CODE:
    
    model_results_combined = {'model':model_name,
                     'model-params':shared_params,
                      'summary':summary_combined,}
                      
    fname_summaries = FPATHS['results']['review-summary-02_json']
    with open(fname_summaries,'w') as f:
        json.dump(model_results_combined, f )

# LLMs

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [ ]:
# # source: https://python.langchain.com/docs/integrations/chat/openai
# template = (
#     "You are a helpful assistant that translates {input_language} to {output_language}."
# )
# system_message_prompt = SystemMessagePromptTemplate.from_template(template)
# human_template = "{text}"
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
# chat_prompt = ChatPromptTemplate.from_messages(
#     [system_message_prompt, human_message_prompt]
# )

# # get a chat completion from the formatted messages
# chat(
#     chat_prompt.format_prompt(
#         input_language="English", output_language="French", text="I love programming."
#     ).to_messages()
# )

In [ ]:

chat = ChatOpenAI(temperature=0)#,api_key=)

In [ ]:
# model_results_combined['summary']

In [ ]:
## Importing the summaris to use as context

with open(FPATHS['results']['review-summary-01_json'],'r') as f:
    summaries = json.load(f)

summaries.keys()

In [ ]:


template_assistant = "You are a helpful assistant data scientist who uses NLP analysis to {task}. {context}."


star_one = summaries['summary-low']
star_five = summaries['summary-high']
context = f"Here is a summary of 1-star reviews: {star_one}.\n\n Here is a summary of 5-star reviews{star_five}"


task_options = {"summarize":'summarize what customers did and did not like about the product.',
               'recommend':'provide a list of 3-5 actionable business recommendations on how to improve the product.'}


# source: https://python.langchain.com/docs/integrations/chat/openai
system_message_prompt = SystemMessagePromptTemplate.from_template(template_assistant)
human_template = "{query}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)                       

In [ ]:
chat_prompt.input_variables

In [ ]:
# query = "What are the results of your analysis?"
# # get a chat completion from the formatted messages
# response= chat.invoke( chat_prompt.format_prompt(query=query, 
#                               context=context, task=task_options['summarize']).to_messages() )

In [ ]:
# print(response.content)

In [ ]:
# # get a chat completion from the formatted messages
# response= chat.invoke( chat_prompt.format_prompt(query=query, 
#                               context=context, task=task_options['recommend']).to_messages() )

In [ ]:
# print(response.content)

### Functionizing It

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI


query = "What are the results of your analysis?"
def get_answer(summaries,selected_task,query):
    
    template_assistant = "You are a helpful assistant data scientist who uses NLP analysis to {task}. {context}."

    star_one = summaries['summary-low']
    star_five = summaries['summary-high']
    context = f"Here is a summary of 1-star reviews: {star_one}.\n\n Here is a summary of 5-star reviews{star_five}"
    
    
    task_options = {"summarize":'summarize what customers did and did not like about the product.',
                   'recommend':'provide a list of 3-5 actionable business recommendations on how to improve the product.'}
    
    
    # source: https://python.langchain.com/docs/integrations/chat/openai
    system_message_prompt = SystemMessagePromptTemplate.from_template(template_assistant)
    human_template = "{query}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )                       
#     return chat_prompt

# def get_answer(chat_prompt, query)

    chat = ChatOpenAI(temperature=0)
    response = chat.invoke( chat_prompt.format_prompt(query=query, 
                                  context=context, task=task_options[selected_task]).to_messages() )
    return response.content

# Generation of Vector Database

## Vector Databases (Making a Chrome dB for Reviews)

In [ ]:
# !pip install chromadb

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
import pandas as pd

In [ ]:
fpath_llm = FPATHS['data']['app']['reviews-with-target-for-llm_csv']
fpath_db = FPATHS['data']['app']['vector-db_dir']

fpath_llm, fpath_db

### Filtering and Saving Review Data for LLM

In [ ]:
fpath_df = FPATHS['data']['processed-nlp']['processed-reviews-with-target_json']
df = pd.read_json(fpath_df)
df.head()

In [ ]:
df['target-rating'].value_counts(dropna=False)

In [ ]:
# df['stars'] = df['overall'].copy()


In [ ]:
df.columns

In [ ]:
# df_llm = df.dropna(subset=['target-rating'])
llm_cols = ['reviewerID','review-text-full','overall']
df_llm = df.loc[:,#df['target-rating'].notna(),
                llm_cols]
df_llm = df_llm.rename({'overall':'stars','review-text-full':'review'},axis=1)
df_llm

In [ ]:
df_llm.duplicated(subset=['review','stars']).sum()

In [ ]:
# 
df_llm = df_llm.drop_duplicates(subset=['review','stars'])
df_llm.to_csv(fpath_llm, index=False)

In [ ]:
import pandas as pd
df = pd.read_csv(fpath_llm)
df['stars'].value_counts()

# 👉🚥 Constructing My App (02/14/24+)

In [ ]:
## Adding caching to reduce api usage
from langchain.cache import InMemoryCache
from langchain.document_loaders import CSVLoader
from langchain.globals import set_llm_cache
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate, PromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.text_splitter import CharacterTextSplitter#, SpacyTextSplitter
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
# from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings

# set_llm_cache(InMemoryCache())

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools.retriever import create_retriever_tool

In [ ]:
##Load in the data
import json

with open("config/filepaths.json") as f:
    FPATHS = json.load(f)

In [ ]:
fpath_llm = FPATHS['data']['app']['reviews-with-target-for-llm_csv']
fpath_db = FPATHS['data']['app']['vector-db_dir']

fpath_llm, fpath_db

In [ ]:
# Load Document --> Split into chunks

loader = CSVLoader(fpath_llm,metadata_columns=['reviewerID'])
documents = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

In [ ]:
print(docs[0].page_content)

In [ ]:
SAVE_EMBEDDINGS = True

In [ ]:
# if SAVE_EMBEDDINGS == True:
#     # Use EMbedding --> embed chunks --> vectors
#     embedding_func = OpenAIEmbeddings()
#     db = Chroma.from_documents(docs, embedding_func, persist_directory= fpath_db)#'./app-assets/reviews_db')
#     # Use persist to save to disk
#     db.persist()
# else:
#     db = Chroma(persist_directory=fpath_db, 
#            embedding_function=OpenAIEmbeddings())
    

In [ ]:
# # Make a retreiver object
# retriever = db.as_retriever(k=6)
# retriever

#### def function to load vector database

In [ ]:
def load_vector_database(fpath_db, fpath_csv=None, metadata_columns = ['reviewerID'],
                         chunk_size=500, use_previous = True,
                         delete=False, as_retriever=False, k=8, **retriever_kwargs):
    
     # Use EMbedding --> embed chunks --> vectors
    embedding_func = OpenAIEmbeddings()
    
    if delete==True:
        # Set use_pervious to False
        use_previous= False
        db = Chroma(persist_directory=fpath_db, 
           embedding_function=embedding_func)
        db.delete_collection()

    if use_previous==True:
        db =  Chroma(persist_directory=fpath_db, 
           embedding_function=embedding_func)
    else:
        if fpath_csv == None:
            raise Exception("Must pass fpath_csv if use_previous==False or delete==True")
                
        # Load Document --> Split into chunks
        loader = CSVLoader(fpath_csv,metadata_columns=metadata_columns)
        documents = loader.load()
        
        text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size)
        docs = text_splitter.split_documents(documents)
        
        db = Chroma.from_documents(docs, embedding_func, persist_directory= fpath_db)
        # Use persist to save to disk
        db.persist()

    if as_retriever:
        return db.as_retriever(k=k, **retriever_kwargs)
    else:
        return db

    
    

In [ ]:
%%time
# Delete previous and make new 
fpath_llm_csv = FPATHS['data']['app']['reviews-with-target-for-llm_csv']
fpath_db = FPATHS['data']['app']['vector-db_dir']
db = fn.load_vector_database( fpath_db,fpath_llm_csv, delete=True)#, use_previous=False)

In [ ]:
%%time
# make retriever for previous
retriever = fn.load_vector_database(fpath_db, delete=False, use_previous=True, as_retriever=True)
retriever

In [ ]:
# dir(db)

In [ ]:
res = db.get()
res.keys()

In [ ]:
res['documents']

In [ ]:
# if len(db.get())>

In [ ]:
db.similarity_search("noodles")

In [ ]:
retriever.get_relevant_documents(query='bad taste')

### New Agent Alternative to ConversationalREtriever
https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents?ref=blog.langchain.dev

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

## Make retreieval tool
tool = create_retriever_tool(
     db.as_retriever(k=6),
    name="search_reviews",
    description="Searches and returns excerpts from Amazon user reviews.")
tools = [tool]

In [ ]:
tool

#### Stealing the Prompt from QA Chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
# from langchain.utils.
llm = ChatOpenAI(temperature=0)
temp_chain = load_qa_chain(llm,)
temp_chain
type(temp_chain)

In [ ]:
temp_chain.llm_chain.prompt

In [ ]:
# Pull starter prompt from langchainhub
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

In [ ]:
prompt.input_schema()

In [ ]:
prompt.input_variables

- Stealing the prompt from load_qa_chain result to prevent erroneous answers.



In [ ]:
qa_prompt_template= "Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"
print(qa_prompt_template)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def load_product_info(fpath=FPATHS['data']['app']['product-metadata-llm_json']):
    with open(fpath,'r') as f:
        product_json = json.load(f)
        
    product_string = "Product Info:\n"
    for k,v in product_json.items():
        if k.lower()=='description':
            continue
        product_string+=f"\n{k} = {v}\n"
        
    return product_string

In [ ]:
help(fn)

In [ ]:
# ## Load in product_json
# with open(FPATHS['data']['app']['product-metadata-llm_json'],'r') as f:
#     product_json = json.load(f)
# product_json

In [ ]:
product_string = fn.app_functions.load_product_info(FPATHS['data']['app']['product-metadata-llm_json'])
product_string

In [ ]:
# # Replace system prompt
# Pull starter prompt from langchainhub
prompt = hub.pull("hwchase17/openai-tools-agent")

# topic =  "answering questions about the product"
# template = f"You are a helpful assistant for {topic} based on the product reviews documents."
# prompt.messages[0] = SystemMessagePromptTemplate.from_template(template)
template = f"""
You are a helpful data analyst for answering questions about the product using the product metadata json:
```{product_string}```\n"""
qa_prompt_template= "Use the results of the documents retreiver to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer:\n----------------\n{agent_scratchpad}"

template+=qa_prompt_template
# Replace system prompt
# topic =  "answering questions about the product"
# template = f"You are a helpful assistant for {topic} based on the product reviews documents."
prompt.messages[0] = SystemMessagePromptTemplate.from_template(template)
# prompt.format(product_metadata=product_json,context=
prompt.input_schema()

In [ ]:
new_prompt = ChatPromptTemplate.from_messages(prompt.messages)
new_prompt.input_schema()

In [ ]:
type(prompt)

In [ ]:
prompt.input_variables

In [ ]:
# sys_template = "You are a helpful assistant for answering questions about the product from the product reviews documents."
# prompt_messages = [
#      SystemMessagePromptTemplate.from_template(sys_template),
#     MessagesPlaceholder(variable_name='chat_history', optional=True),
#     HumanMessagePromptTemplate(prompt=PromptTemplate.from_template("{input}")),
#     MessagesPlaceholder(variable_name='agent_scratchpad')
# ]
# prompt_messages

In [ ]:
# prompt_manual = ChatPromptTemplate.from_messages(prompt_messages)

In [ ]:
# type(prompt),type(prompt_manual)

In [ ]:
# prompt.messages, prompt_manual.messages

In [ ]:
# prompt.format_prompt(topic="pretending to be a low-carb consumer with opinions derived from ")

In [ ]:
# prompt.input_variables, prompt_manual.input_variables

In [ ]:
llm = ChatOpenAI(temperature=0)
agent = create_openai_tools_agent(llm, tools, new_prompt,)#prompt_manual)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,
                               memory=ConversationBufferMemory())#[],memory_key='chat_history'))
agent_executor

In [ ]:
agent_executor.input_keys

In [ ]:
# agent_executor.get_lc_namespace()

In [ ]:
## Using the agent
q = "Hello, there!"
result  = agent_executor.invoke(dict(input=q))
result.keys()

```python
KeyError:"Input to ChatPromptTemplate is missing variables {'search_reviews'}.  Expected: ['agent_scratchpad', 'input', 'search_reviews'] Received: ['input', 'history', 'intermediate_steps', 'agent_scratchpad']"
```

In [ ]:
result['output']

In [ ]:
q= "How do these noodles compare to other low-carb noodles in terms of taste?"
# result = agent_executor.invoke(dict(input=q))
# print(result['output'])

In [ ]:
q= "What do glute-free customers think of these?"
# result = agent_executor.invoke(dict(input=q))
# print(result['output'])

In [ ]:
q= "How about cook time?"
# result = agent_executor.invoke(dict(input=q))
# print(result['output'])

In [ ]:
# q= "cooking time?"
# result = agent_executor.invoke(dict(input=q))
# print(result['output'])

In [ ]:
# df_llm['review'].str.contains("cooking time").sum()

In [ ]:
# db.similarity_search(query="cooking time", k=8)

In [ ]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt

In [ ]:
prompt.messages

> - Trying to address agent using the results in the prompt: https://github.com/langchain-ai/langchain/issues/14209#issuecomment-1851352078

In [ ]:
import langchain
from langchain.agents.initialize import initialize_agent
# initialize_agent(tools=tools, llm=llm, )
# dir(langchain.agents.initialize)

In [ ]:
# db = Chroma(persist_directory=fpath_db, 
#                embedding_function=OpenAIEmbeddings())

fpath_llm_csv = FPATHS['data']['app']['reviews-with-target-for-llm_csv']
fpath_db = FPATHS['data']['app']['vector-db_dir']
db = fn.load_vector_database( fpath_db,fpath_llm_csv, delete=True)#, use_previous=False)
def get_agent(fpath_db, k=8, temperature=0.1,
             return_messages=True, verbose=False):
    
    
    # import custom_functions as fn
    from custom_functions.app_functions import load_product_info
    product_string = load_product_info(FPATHS['data']['app']['product-metadata-llm_json'])
    ## Make retreieval tool
    tool = create_retriever_tool(
         db.as_retriever(k=k),
        "search_reviews",
        "Searches and returns excerpts from Amazon user reviews.",
    )
    tools = [tool]

    # Pull starter prompt from langchainhub
    prompt = hub.pull("hwchase17/openai-tools-agent")

    # produt_string = 
    # # Replace system prompt
    template = f"You are a helpful data analyst for answering questions about what customers said about a specific  Amazon product using only content from use reviews."
    product_template = f" Assume all user questions are asking about the content in the user reviews. Note the product metadata is:\n```{product_string}```\n\n"
    template+=product_template
    
    # template+="\n\nUse information from the following review documents to answer questions:"
    # qa_prompt_template= "\n- Here are the review documents:\n----------------\n{agent_scratchpad}\n\n"
    qa_prompt_template ="""Use the following pieces of context (user reviews) to answer the user's question by summarizing the reviews. 
            If you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{agent_scratchpad}\n\n"""
    template+=qa_prompt_template
    # template+="Try to infer one based on the review documents, otherwise just say that you don't know, don't try to make up an answer"

    # Replace system prompt
    prompt.messages[0] = SystemMessagePromptTemplate.from_template(template)
    prompt = ChatPromptTemplate.from_messages(prompt.messages)

    if verbose:
        print(prompt.messages)
        
    llm = ChatOpenAI(temperature=temperature)
    agent = create_openai_tools_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, 
                                   memory=ConversationBufferMemory(return_messages=return_messages))
    return agent_executor

In [ ]:
agent_exe = get_agent(fpath_db, k=8,verbose=False)

type(agent_exe.agent)

In [ ]:
q = "What is the cooking time of the product?"
response = agent_exe.invoke({"input":q})
print(response['output'])

In [ ]:
db.similarity_search('cook time')

In [ ]:
q = "How many reviews mentioned the cooking time?"
response = agent_exe.invoke({"input":q})
print(response)

In [ ]:
q = "What did customers say about cooking time?"
response = agent_exe.invoke({"input":q})
print(response)

#  BOOKMARK 02/15/24: Improving detailed queries

In [ ]:
def get_agent_v1(fpath_db, k=8, temperature=0.1,topic =  "answering questions about the product",
             return_messages=True):
    
    ## Make retreieval tool
    tool = create_retriever_tool(
         db.as_retriever(k=k),
        "search_reviews",
        "Searches and returns excerpts from Amazon user reviews.",
    )
    tools = [tool]
    # Pull starter prompt from langchainhub
    prompt = hub.pull("hwchase17/openai-tools-agent")
    # Update starter prompt 
    template = f"You are a helpful assistant for {topic} based on the Amazon product review documents. Include quotes from the documents, when appropriate."
    template+=f"Here is some additional metadata about the product for your reference: ```{product.to_string()}```"
    # template = "You are a helpful assistant for answering questions about the product from the product reviews documents."
    prompt.messages[0] = SystemMessagePromptTemplate.from_template(template)
    prompt = ChatPromptTemplate.from_messages(prompt.messages)
    # prompt.messages[0] = prompt.messages[0].format_messages(topic=topic)

    llm = ChatOpenAI(temperature=0)
    agent = create_openai_tools_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               memory=ConversationBufferMemory(return_messages=return_messages))
    return agent_executor


# def get_agent_v1(fpath_db, k=6, temperature=0.1,topic =  "answering questions about the product",
#              return_messages=True):
    
#     db = Chroma(persist_directory=fpath_db, 
#            embedding_function=OpenAIEmbeddings())
    
#     ## Make retreieval tool
#     tool = create_retriever_tool(
#          db.as_retriever(k=k),
#         "search_reviews",
#         "Searches and returns excerpts from Amazon user reviews.",
#     )
#     tools = [tool]
#     # Pull starter prompt from langchainhub
#     prompt = hub.pull("hwchase17/openai-tools-agent")
#     # Update starter prompt 
#     template = f"You are a helpful assistant for {topic} based on the Amazon product review documents."
#     # template = "You are a helpful assistant for answering questions about the product from the product reviews documents."
#     prompt.messages[0] = SystemMessagePromptTemplate.from_template(template)
#     # prompt.messages[0] = prompt.messages[0].format_messages(topic=topic)

#     llm = ChatOpenAI(temperature=0)
#     agent = create_openai_tools_agent(llm, tools, prompt)
#     agent_executor = AgentExecutor(agent=agent, tools=tools, 
#                                memory=ConversationBufferMemory(return_messages=return_messages))
#     return agent_executor


agent_exe = get_agent_v1(fpath_db,k=8)

In [ ]:
agent_exe

In [ ]:
q = "How is the cooking time?"
response = agent_exe.invoke({"input":q})
print(response['output'])

In [ ]:
q = "Provide a summary list of what customers who rated the product as 1 or 2 stars did not like vs what the customers who gave it 5 stars did like"
response = agent_exe.invoke({'input':q})
print(response['output'])

In [ ]:
response['history'][:-1]

In [ ]:
response = agent_exe.invoke({'input':'What do you recommend the company address to make customers happier?'})
print(response['output'])

In [ ]:
# response = agent_exe.invoke({'input':'What do you recommend the company address to make customers happier?'})

In [ ]:
agent_exe = get_agent(fpath_db)
q = "Please summarize what consumers who gave it a Low rating did not like about the product"
response = agent_exe.invoke({'input':q})
print(response['output'])

In [ ]:
response['history']

In [ ]:

# agent_cust_no_carb = get_agent(fpath_db, 
#                                # topic = "act as if you were a strict low-carb consumer with base your opinions and word choices")
# response = agent_cust_no_carb.invoke({'input':'What was the?'})
# print(response['output'])

In [ ]:
flavors = 

In [ ]:
# # Chain 1 
# # take query from user --> generate multi queries 
# llm  = ChatOpenAI(temperature=0)

# chain1  = load_qa_chain(llm, chain_type='stuff', verbose=True)
# chain1.input_keys

In [ ]:
# question= "I've tried so many other low carb noodles how does this one compare?"
# # relevant_docs = db_connection.as_retriever().get_relevant_documents(question)
# # len(relevant_docs)

# # Chain 2 
# # take multi queries --> get relevant documents


# # Chain 3
# # take relevant documents --> Summarize relevant documents

# # Chain 4
# # summary --> recommendations

# 📝 NOTES BELOW: LangChain Course

In [ ]:
# Load Document --> Split into chunks
loader = CSVLoader(fpath_llm, metadata_columns=['reviewerID'])
documents = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

In [ ]:
docs[0]

In [ ]:
SAVE_EMBEDDINGS = True

# Use EMbedding --> embed chunks --> vectors
embedding_func = OpenAIEmbeddings()
# Vector-chuinks -> save chromadb
if SAVE_EMBEDDINGS:
    db = Chroma.from_documents(docs, embedding_func, persist_directory= fpath_db)#'./app-assets/reviews_db')
    
    # Use persist to save to disk
    db.persist()

In [ ]:
# Load from disk
db_connection = Chroma(persist_directory=fpath_db,
                      embedding_function=OpenAIEmbeddings())
db_connection

In [ ]:
# query = "I've tried so many other low carb noodles how does this one compare?"
query = 'How long do they take to cook?'

In [ ]:
similar_docs = db_connection.similarity_search(query)
len(similar_docs)

In [ ]:
# relelvant content
doc_content = [doc.page_content for doc in similar_docs]
doc_content

### Document Retreivers

> Made from vector dbs. Adds new methods (used internally)

In [ ]:
retriever = db_connection.as_retriever()
retriever

In [ ]:
relevant_docs = retriever.get_relevant_documents(query)
len(relevant_docs)

In [ ]:
## Can use MultiQueryRetreiver to make variants of initial query
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai.chat_models import ChatOpenAI

In [ ]:
## Adding a new document
query = "I've tried so many other low carb noodles how does this one compare?"
query

In [ ]:
chat = ChatOpenAI()
retreiever_from_llm = MultiQueryRetriever.from_llm(retriever=db_connection.as_retriever(),
                                                  llm=chat)
print(retreiever_from_llm)

In [ ]:
docs_multi_query = retreiever_from_llm.get_relevant_documents(query=query)
len(docs_multi_query)

In [ ]:
# docs_multi_query.

In [ ]:
# relelvant content
doc_content_multi = [doc.page_content for doc in docs_multi_query]
doc_content_multi

### Chains

In [ ]:
## Adding caching to reduce api usage
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
# set_llm_cache(InMemoryCache())

In [ ]:
from langchain.chains.question_answering import load_qa_chain
# from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
query

In [ ]:
llm  = ChatOpenAI(temperature=0)
chain  = load_qa_chain(llm, chain_type='stuff')
question= "I've tried so many other low carb noodles how does this one compare?"
relevant_docs = db_connection.as_retriever().get_relevant_documents(question)
len(relevant_docs)

In [ ]:
answer = chain.run(input_documents=relevant_docs, question=question)
answer

In [ ]:
# chain  = load_qa_with_sources_chain(llm, chain_type='stuff')
# answer = chain.run(input_documents=relevant_docs, question=question)
# answer

#### Memories

In [ ]:
from langchain.memory import ChatMessageHistory

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.llms import OpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import os

## LLM Completions

In [ ]:
llm = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
llm

In [ ]:
starter = "The reason that I loved these Miracle Noodles spinach-based noodles was "
# llm.invoke(starter)

## PromptTemplates

In [ ]:
from langchain import PromptTemplate

In [ ]:
# General Template with no inputs
no_input_prompt = PromptTemplate(input_variables=[],
                                template="Tell me a fact:")
no_input_prompt.format()

In [ ]:
# llm.invoke(no_input_prompt.format())

In [ ]:
single_input_prompt = PromptTemplate(input_variables=['topic'],
                                template="Tell me a fact about {topic}")
single_input_prompt.format(topic='Mars')

In [ ]:
# llm.invoke(single_input_prompt.format(topic='Mars'))

In [ ]:
multi_input_prompt = PromptTemplate(input_variables=['topic','level','person'],
                                template="Tell me a fact about {topic} for a {level} {person}")
# llm.invoke(multi_input_prompt.format(topic='Mars', level='PhD', person='advisor'))

### Chat Models

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

In [ ]:
chat =  ChatOpenAI(api_key=os.environ['OPENAI_API_KEY'])
# chat.invoke(starter)

In [ ]:
# Flexible starting template
system_template = "You are an AI recipe assistant that specializes in {dietary_preference} dishes that can be prepared in {cook_time}"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
system_message_prompt.input_variables

In [ ]:
human_template=  "{recipe_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
human_message_prompt.input_variables

In [ ]:
chat_prompt  = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt.input_variables

In [ ]:
# Run format_prompt to pass in all inputs
prompt= chat_prompt.format_prompt(cook_time='60 min',
                                  dietary_preference='gluten free', 
                                  recipe_request="Quick Snack").to_messages()
prompt

In [ ]:
# # result =  chat.invoke(prompt)
# print(result.content)

### ~~FewShotPrompt templates~~

In [ ]:
# from langchain.prompts.chat import AIMessagePromptTemplate

### ~~Serialization of Prompts~~ 

In [ ]:
# chat_prompt.save('example.json')

### LangChain Document Loaders

In [ ]:
from langchain.document_loaders import CSVLoader

In [ ]:
# ADMIN PREP
# FPATHS['data']['processed-nlp']['processed-reviews-with-target_json']
fpath_llm = FPATHS['data']['app']['reviews-with-target-for-llm_csv']

In [ ]:
# df = pd.read_json(FPATHS['data']['processed-nlp']['processed-reviews-with-target_json'])
df = pd.read_csv(FPATHS['data']['app']['reviews-with-target-for-llm_csv']) 
df

In [ ]:
loader = CSVLoader(fpath_llm)

data = loader.load()
type(data)
                   

In [ ]:
len(data)

In [ ]:
type(data[0])

In [ ]:
print(data[0].page_content)

In [ ]:
print(data[0].metadata)

### Document Transformers

> Vectorize text for documents

In [ ]:
df

In [ ]:
import seaborn as sns

In [ ]:
char_count = df['review-text-full'].map(lambda x: len(x))
sns.histplot(char_count)

In [ ]:
token_lengths = df['review-text-full'].map(lambda x: len(x.split(" ")))
sns.histplot(token_lengths)

In [ ]:
example_text = df.loc[token_lengths.idxmax(),'review-text-full']
print(len(example_text.split(" ")))
example_text[:1000]

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(separator="\n\n",chunk_size=1000)
text_splitter

In [ ]:
# Create documents
texts = text_splitter.create_documents([example_text])
type(texts)

In [ ]:
len(texts)

In [ ]:
texts[0]

In [ ]:
# OpenAI's package for tokenization (offline)
# !pip install tiktoken

In [ ]:
# Use split_text instead of create_documents
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
texts = text_splitter.split_text(example_text)
len(texts)

In [ ]:
texts[0]

### Creating Embeddings for Vectorized Database

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

In [ ]:
embeddings =  OpenAIEmbeddings(model='text-embedding-ada-002') #default
embeddings

In [ ]:
embedded_text = embeddings.embed_query(example_text)
len(embedded_text)

In [ ]:
embedded_text[:5]

## Vector Databases (Making a Chrome dB for Reviews)

In [ ]:
# !pip install chromadb

In [ ]:

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS

In [ ]:
# # Load Document --> Split into chunks
# loader = CSVLoader(fpath_llm)
# documents = loader.load()

# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
# docs = text_splitter.split_documents(documents)

In [ ]:
# docs[0]

In [ ]:
fpath_db = FPATHS['data']['app']['vector-db_dir']
fpath_db

In [ ]:
# os.makedirs("./app-assets/reviews-db/", exist_ok=True)

In [ ]:

# # Use EMbedding --> embed chunks --> vectors
# embedding_func = OpenAIEmbeddings()
# # Vector-chuinks -> save chromadb

# db = Chroma.from_documents(docs, embedding_func, persist_directory= fpath_db)#'./app-assets/reviews_db')
# db.persist()


# # query --> similarity search chromadb

In [ ]:
# Load from disk
db_connection = Chroma(persist_directory=fpath_db,
                      embedding_function=OpenAIEmbeddings())
db_connection

In [ ]:
query = "I've tried so many other low carb noodles how does this one compare?"

In [ ]:
similar_docs = db_connection.similarity_search(query)
len(similar_docs)

In [ ]:
# relelvant content
doc_content = [doc.page_content for doc in similar_docs]
doc_content

### Document Retreivers

> Made from vector dbs. Adds new methods (used internally)

In [ ]:
retriever = db_connection.as_retriever()
retriever

In [ ]:
relevant_docs = retriever.get_relevant_documents(query)
len(relevant_docs)

In [ ]:
## Can use MultiQueryRetreiver to make variants of initial query
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai.chat_models import ChatOpenAI

In [ ]:
## Adding a new document
query = "I've tried so many other low carb noodles how does this one compare?"
query

In [ ]:
chat = ChatOpenAI()
retreiever_from_llm = MultiQueryRetriever.from_llm(retriever=db_connection.as_retriever(),
                                                  llm=chat)
print(retreiever_from_llm)

In [ ]:
docs_multi_query = retreiever_from_llm.get_relevant_documents(query=query)
len(docs_multi_query)

In [ ]:
docs_multi_query

In [ ]:
# relelvant content
doc_content_multi = [doc.page_content for doc in docs_multi_query]
doc_content_multi

### Answer Compression

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
## Define llm 
llm = ChatOpenAI(temperature=0)
# insert llm into an llm chain extractor
compressor = LLMChainExtractor.from_llm(llm)

# use chain extractor inside context compression extractor
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, 
                                                      base_retriever=db_connection.as_retriever())
compression_retriever

In [ ]:
query

In [ ]:
normal_retriever = db_connection.as_retriever()
normal_docs = normal_retriever.get_relevant_documents(query)
len(normal_docs)

In [ ]:
normal_docs[0]

In [ ]:
compressed_docs = compression_retriever.get_relevant_documents(query)
len(compressed_docs)

In [ ]:
normal_docs[0]

In [ ]:
compressed_docs[0]

In [ ]:
compressed_docs[0].metadata#['summary']

### Chains

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
query

In [ ]:
llm  = ChatOpenAI(temperature=0)
chain  = load_qa_chain(llm, chain_type='stuff')
question= "I've tried so many other low carb noodles how does this one compare?"
relevant_docs = db_connection.as_retriever().get_relevant_documents(question)
len(relevant_docs)

In [ ]:
answer = chain.run(input_documents=relevant_docs, question=question)
answer

In [ ]:
chain  = load_qa_with_sources_chain(llm, chain_type='stuff')
answer = chain.run(input_documents=relevant_docs, question=question)
answer

#### Memories

In [ ]:
from langchain.memory import ChatMessageHistory

### LLMChain Object (02/24/24)


In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

In [ ]:
human_template = "Make up a funny name for a company that makes {product}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat = ChatOpenAI(temperature=0)

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([human_prompt])

In [ ]:
from langchain.chains import LLMChain
# LLM Chain takes 2 args
# llm to connect to
# prompts for that model
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
chain.input_keys

- Chains return  just a string

In [ ]:
# Use chain.invoke and pass in args for template
result = chain.invoke(input=dict(product="Computers"))
result

### SimpleSequentialChain

- SimpleSequentialChain can only do 1 input/output

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0)

# Chain 1 - blog post outline
template1 = "Give me a simple bullet point outline for a blog post on {topic}"
first_prompt = ChatPromptTemplate.from_template(template1)
chain_1 = LLMChain(llm=llm, prompt = first_prompt)

# chain 2 -  write blog post
template2 = "Write a full blog post using this outline: {outline}"
second_prompt = ChatPromptTemplate.from_template(template2)
chain_2 =  LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
## Create full chain

full_chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)
full_chain

In [ ]:
result = full_chain.invoke(input="Large Language Models")
result

In [ ]:
print(result['output'])

### SequentialChain

- more than 1 input/output

In [ ]:
from langchain.chains import SequentialChain, LLMChain,SimpleSequentialChain
llm = ChatOpenAI(temperature=0)

## Employee Performance Review  INPUT TEXT

## review_text --> LLMCHAIN --> Summary 
template1 = "Give a summary of this employee's performance review:\n {review} "
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = LLMChain(llm=llm, prompt=prompt1, 
                  output_key='review_summary' # Name for output saved in dict
                                              # (MUST MATCH THE INPUT NAME FOR NEXT CHAIN!)
                 )


## Summary --> LLMChain --> weaknesses
template2 = "Identify a list of key employee weaknesses in this review summary: {review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain2 = LLMChain(llm=llm, prompt=prompt2,  output_key='weaknesses')


# weaknesses --> LLMCain --> improvement plan
template3 = "Create a personalized plan to help address and fix these weaknesses: {weaknesses}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain3 = LLMChain(llm=llm, prompt=prompt3,  output_key='final_plan')

In [ ]:
full_chain  = SequentialChain(chains=[chain1, chain2, chain3], 
                              input_variables=['review'], # very first input
                              output_variables=['review_summary', # should match the outputs of each chain
                                                'weaknesses', # best practice is to include all outputs
                                                'final_plan'
                                               ],                                               
                              verbose=True )

In [ ]:
from pathlib import Path
review = Path('data/raw/fake-employee-review.md').read_text()
review[:1000]

In [ ]:
result = full_chain.invoke(review)
result.keys()

In [ ]:
# result.keys()

In [ ]:
print(result['weaknesses'])

In [ ]:
print(result['final_plan'])

# Adding My Apps Usage

- Construct message histories with my prompts?

- flavor:
    - summary, customer
- type (summary):
    - What they did/didn't like
    - recommendations for improving product
    - recommendations for marketing
- type (customer):
    - low-carb, general

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma#, FAISS
from langchain.memory import ChatMessageHistory, ConversationSummaryBufferMemory, ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

In [ ]:
# chat_history = ChatMessageHistory(messages=[])
# chat_history

In [ ]:
## Set up conversation chain with memory

llm = OpenAI()
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, 
                                 memory=memory,
                                 verbose=True)

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage


In [ ]:
# PromptTemplate()

In [ ]:
# ChatPromptTemplate()

In [ ]:
flavor_options = {
    "Summary(General)": "You are a helpful data scientist presenting your findings to a non-technical CEO to summarize what customers do and do not like.",
    # "Summary(Bartender)": "You are a charming and emotionally intelligent bartender who gives great advice. You annotate your physical actions with new lines and asterisks as you answer. Act as helpful assistant who uses ngrams from product reviews to summarize that customers do and do not like.",
    "Customer (Low Carb/Gluten Free)": "You are an average American low-carb-diet or gluten-free consumer. You're used to how different non-traidtional grains and pastas can be.",
    "Customer (General)":  "You are an average American consumer who does not follow a special diet. You are used to traditional grains and pastas."
}

selected_flavor = "Summary(General)"
assistant_type = flavor_options[selected_flavor]
assistant_type

# flavor_messages  = {}
# for name, prompt in flavor_options.items():
#     flavor_messages[name] = [SystemMessage(prompt)]
# ]

In [ ]:

def create_conversation(assistant_type,):
    # Add the rest of the prompt
    template_starter = assistant_type
    template = template_starter + """
    Current conversation:
    {history}
    Human: {input}
    AI Assistant:"""
    llm = ChatOpenAI(temperature=0)
    PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
    conversation = ConversationChain(
        prompt=PROMPT,
        llm=llm,
        verbose=True,
        memory=ConversationBufferMemory(ai_prefix="AI Assistant"), #SummaryMemory?
    )

    return conversation

conversation = create_conversation(flavor_options['Summary(General)'])
conversation

In [ ]:
conversation.memory.buffer_as_messages

In [ ]:
# question = "Based "

In [ ]:
# response=st.session_state['conversation'].predict(input=query)


In [ ]:

# def set_conversation_flavor(llm,flavor_name):
#     # Select the correct prompt from the dictionary of options
#     flavor= flavor_options[flavor_name]
 
#     # Use an f-string to constuct the new start of prompt
#     flavor_text = f"{assistant_type}"
#     # Add the rest of the prompt
#     template = flavor_text + """
#     Current conversation:
#     {history}
#     Human: {input}
#     AI Assistant:"""
#     PROMPT = PromptTemplate(input_variables=["assistant_type","history", "input"], template=template)
#     conversation = ConversationChain(
#         prompt=PROMPT,
#         llm=llm,
#         verbose=True,
#         memory=ConversationBufferMemory(ai_prefix="AI Assistant"), #SummaryMemory?
#     )
#     return conversation


In [ ]:
messages  = [
    SystemMessage(content="You are a data scientist presenting your findings to a non-technical CEO."),
    # HumanMessage()
]
result = chat.generate()

In [ ]:
# ngrams_df = 

## Models App

# PREVIOUS WORK

## ChatGPT Interpretation - Using ngrams

In [ ]:
# import time,os
# # from streamlit_chat

# ## LLM Classes 
# from langchain_openai import OpenAI
# # from langchain_openai.chat_models import ChatOpenAI
# from langchain.chains import ConversationChain
# from langchain.schema import HumanMessage, SystemMessage, AIMessage


# ## Memory Modules
# from langchain.chains.conversation.memory import (ConversationBufferMemory, 
#                                                   ConversationSummaryBufferMemory,
#                                                   ConversationBufferWindowMemory,
#                                                   ConversationSummaryMemory)
# # Template for changing conversation chain's "flavor"
# from langchain.prompts.prompt import PromptTemplate


In [ ]:
# from PIL import Image
# # img = Image.open('images/OpenAI_Logo.svg')
# img

In [ ]:
# # Create required session_state containers
# if 'messages' not in st.session_state:
#     st.session_state.messages=[]
    
# if 'API_KEY' not in st.session_state:
#     st.session_state['API_KEY'] = os.environ['OPENAI_API_KEY'] # Could have user paste in via sidebar

# if 'conversation' not in st.session_state:
#     st.session_state['conversation'] = None


In [ ]:
# def reset():
#     if 'messages' in st.session_state:
#         st.session_state.messages=[]

#     if 'conversation' in st.session_state:
#         st.session_state['conversation'] = None


In [ ]:
def get_response(query):
    
    if st.session_state['conversation'] is None:
        llm = OpenAI(max_tokens=500,
            openai_api_key=st.session_state['API_KEY'],
               temperature=float(temp),
            model_name='gpt-3.5-turbo-instruct'  # 'text-davinci-003' model is depreciated now, so we are using the openai's recommended model
        )
  
    
    if st.session_state['conversation'] is None:
        st.session_state['conversation'] = set_conversation_flavor(llm,flavor_name=flavor)

    response=st.session_state['conversation'].predict(input=query)
    # st.session_state['messages'].append()
    print(st.session_state['conversation'].memory.buffer)

    return response
    # return show_history()



def set_conversation_flavor(llm,flavor_name):
    # Select the correct prompt from the dictionary of options
    flavor= flavor_options[flavor_name]
 
    # Use an f-string to constuct the new start of prompt
    flavor_text = f"The following is a conversation between a human and an assistant. The assistant is {flavor}."
    # Add the rest of the prompt
    template = flavor_text + """
    Current conversation:
    {history}
    Human: {input}
    AI {flavor}:"""
    PROMPT = PromptTemplate(input_variables=["flavor","history", "input"], template=template)
    conversation = ConversationChain(
        prompt=PROMPT,
        llm=llm,
        verbose=True,
        memory=ConversationBufferMemory(ai_prefix="AI Assistant"), #SummaryMemory?
    )
    return conversation

In [ ]:
flavor_options = {
    "Summary(General)": "a helpful data analyst who uses ngrams from product reviews to summarize that customers do and do not like.",
    "Summary(Bartender)": " a charming and emotionally intelligent bartender who gives great advice. You annotate your physical actions with new lines and asterisks as you answer. Act as helpful assistant who uses ngrams from product reviews to summarize that customers do and do not like.",
    "Customer (Low Carb/Gluten Free)": "a typical consumer who follows a low carb diet and has gluten sensitivity. You know what things you like in your food products.",
    "Customer (Genercal)":  "a typical consumer who does not follow a special diet and enjoys eating gluten-containing foods. You know what things you like in your food products.",
}

In [ ]:
flavor_name = st.sidebar.selectbox("Which type of chatbot?", key='no_reset',options=list(flavor_options.keys()), index=0,)
temp=st.sidebar.slider("model temperature:",min_value=0.0, max_value=2.0, value=0.7, step=.1)

llm = OpenAI(max_tokens=1000,
        openai_api_key=os.environ['OPENAI_API_KEY'],
           temperature=float(temp),
        model_name='gpt-3.5-turbo-instruct'
    )

# Select the correct prompt from the dictionary of options
flavor= flavor_options[flavor_name]

# Use an f-string to constuct the new start of prompt
flavor_text = f"The following is a conversation between a human and an assistant. The assistant is {flavor}."
# Add the rest of the prompt
template = flavor_text + """
Current conversation:
{history}
Human: {input}
ChatGPT:"""
PROMPT = PromptTemplate(input_variables=["flavor","history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="ChatGPT"), #SummaryMemory?
)



In [ ]:
def format_ngrams_for_chat(top_n_group_ngrams):
        
    string_table = []
    
    for group_name in top_n_group_ngrams.columns.get_level_values(0).unique():
        print(group_name)
        group_df = top_n_group_ngrams[group_name].copy()
        group_df['Rating Group'] = group_name 
        group_df = group_df.set_index("Rating Group")
        string_table.append(group_df)
        # string_table.append((group_df.values))
    return pd.concat(string_table)

In [ ]:
## Special form of ngrams for chatgpt
chatgpt_stopwords = [*stopwords_list, 'angel','hair','miracle','noodle','shirataki','pasta']
top_n_group_ngrams = fn.show_ngrams(df, top_n=25,ngrams=4, text_col_selection='review-text-full',
                                     stopwords_list=chatgpt_stopwords)
md_table = format_ngrams_for_chat(top_n_group_ngrams)
table_message = f"Heres a table of the most common ngrams from Low Rating reviews and high rating reviews. ```{md_table}```" # Please give me a summary list of what customers liked  and did not like about the product."


In [ ]:
# top_n_group_ngrams = fn.show_ngrams(df, top_n=25,ngrams=4, text_col_selection='review-text-full',
#                                      stopwords_list=chatgpt_stopwords)
# md_table = format_ngrams_for_chat(top_n_group_ngrams)

- Make this message below (wtihout the question). one of the pre-filled in human messages.

In [ ]:
conversation.memory.buffer

In [ ]:
conversation.input_keys

In [ ]:
table_message = f"Heres a table of the most common ngrams from Low Rating reviews and high rating reviews. ```{md_table}```"
conversation.prep_inputs(table_message)

In [ ]:
question = "Please give me a summary list of what customers liked  and did not like."
# resp = conversation.invoke(question)
# resp = conversation.predict(input=question)

# print(resp['response'])

In [ ]:
# resp

In [ ]:
# question = f"Heres a table of the most common ngrams from Low Rating reviews and high rating reviews. ```{md_table}``` Please give me a summary list of what customers liked  and did not like about the product."
# resp = conversation.invoke(question)

# print(resp['response'])

In [ ]:
# list(conversation.memory

In [ ]:
# print(resp['response'])

In [ ]:
# "\n".join(string_table)

In [ ]:
# csv_vals_Low = top_100_group_ngrams['Low'].to_csv()

In [ ]:
# conversation.predict?